### Exercise 1

- Define grammar that covers the following sentences.

    - show flights from new york to los angeles
    - list flights from new york to los angeles
    - show flights from new york
    - list flights to los angeles
    - list flights
    
- Use one of the parsers to parse the sentences (i.e. test your grammar)


In [ ]:
# As reference  
# rules = """
#     'saw' -> 'i' | 'man' | 'with'
#     'man' ->  'telescope' | 'the' | 'with'
#     'telescope' -> 'the' | 'with' | 'a'
# """

sentences = ['show flights from new york to los angeles', 
             'show flights from los angeles to new york'
             'list flights from new york to los angeles',
            'show flights from new york', 
             'list flights to los angeles', 
             'list flights']
rules = """
    'show' -> 'flights' 
    'list' -> 'flights'
    'flights'  -> 'york' | 'angeles'
    'york' -> 'new' | 'from' | 'to'
    'angeles' -> 'los' | 'from' | 'to'
"""

toy_grammar = nltk.DependencyGrammar.fromstring(rules)

np_parser = nltk.ProjectiveDependencyParser(toy_grammar)
for sent in sentences:
    for graph in np_parser.parse(sent.split()):
        print("Sentence:", sent)

        if type(graph) != nltk.tree.Tree:
            graph.tree().pretty_print(unicodelines=True, nodedist=4)
            print("The ROOT is '{}'".format(s_bold + graph.root['word'] + e_bold), '\n')
        else:
            graph.pretty_print(unicodelines=True, nodedist=4)
            print("The ROOT is '{}'".format(s_bold + graph.label() + e_bold), '\n')
        

#### Exercise 2

Write a function that given a dependency graph, for each token (word), produces list of words from it to ROOT.

(Construct normal `dict` for simplicity first.)

In [4]:
from pprint import pprint
# With .nodes we get a dict
dg_tree = dependency_treebank.parsed_sents()[0].tree()
dg = dependency_treebank.parsed_sents()[0].nodes
# Let's print to see what it contains
#pprint(dg)
print(dg_tree.pretty_print(unicodelines=True, nodedist=4))

def go_to_root(token, head, dg):
    end = False
    path = [token]
    if head == 0:
        return path
    while not end:
        next_token = dg[head]['word']
        head = dg[head]['head']
        path.append(next_token)
        if head == 0:
            end = True
    return path

for k, v in sorted(dg.items()):
    if k != 0:
        print(v['word'],":" ,go_to_root(v['word'], v['head'], dg))
        
    

                                   will                                                     
 ┌─────────────────┬────────────────┴────────────────┐                                          
 │               Vinken                             join                                    
 │       ┌─────────┼────────┬───────┐        ┌───────┴─────────┬─────────────────────────┐      
 │       │         │        │      old       │                 as                        │  
 │       │         │        │       │        │                 │                         │      
 │       │         │        │     years    board            director                    Nov.
 │       │         │        │       │        │       ┌─────────┴─────────────┐           │      
 .     Pierre      ,        ,       61      the      a                  nonexecutive     29 

None
Pierre : ['Pierre', 'Vinken', 'will']
Vinken : ['Vinken', 'will']
, : [',', 'Vinken', 'will']
61 : ['61', 'years', 'old', 'Vinken', 'will']
y

### Exercise 3
- Train `arc-standard` and `arc-eager` transition parsers on the same portion (slightly bigger than 100, otherwise it takes a lot of time)
- Evaluate both of them comparing the attachment scores

In [8]:
tp = TransitionParser('arc-standard')
tp.train(dependency_treebank.parsed_sents()[:150], 'tp.model', verbose=False)

parses = tp.parse(dependency_treebank.parsed_sents()[-150:], 'tp.model')
print(dependency_treebank.parsed_sents()[-1:])
de = DependencyEvaluator(parses, dependency_treebank.parsed_sents()[-150:])
las, uas = de.eval()

# no labels, thus identical
print(las)
print(uas)

tp = TransitionParser('arc-eager')
tp.train(dependency_treebank.parsed_sents()[:150], 'tp.model', verbose=False)

parses = tp.parse(dependency_treebank.parsed_sents()[-150:], 'tp.model')
de = DependencyEvaluator(parses, dependency_treebank.parsed_sents()[-150:])
las, uas = de.eval()

# no labels, thus identical
print(las)
print(uas)

 Number of training examples : 150
 Number of valid (projective) examples : 150
[<DependencyGraph with 16 nodes>]
0.7979524239686842
0.7979524239686842
 Number of training examples : 150
 Number of valid (projective) examples : 150
0.8027702499247215
0.8027702499247215


In [9]:
parses = tp.parse(dependency_treebank.parsed_sents()[-1:], 'tp.model')
parses[0].tree().pprint()

(said
  Trinity
  (plans
    it
    (begin to delivery (in (quarter the first (of (year next))))))
  .)
